In [1]:
from __future__ import print_function

import time
import numpy as np

import scipy.misc
import scipy.io

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.models import Model
from keras.layers import Input
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

%matplotlib inline

Using TensorFlow backend.


In [71]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()
K.set_session(sess)

In [72]:
import h5py

f = h5py.File("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5", "r")
print(f.keys())

[u'block1_conv1', u'block1_conv2', u'block1_pool', u'block2_conv1', u'block2_conv2', u'block2_pool', u'block3_conv1', u'block3_conv2', u'block3_conv3', u'block3_conv4', u'block3_pool', u'block4_conv1', u'block4_conv2', u'block4_conv3', u'block4_conv4', u'block4_pool', u'block5_conv1', u'block5_conv2', u'block5_conv3', u'block5_conv4', u'block5_pool', u'input_2']


In [73]:
STYLE_LAYERS = [
    ('block1_conv1', 0.2),
#     ('block1_conv2', 0.2),
#     ('block2_conv1', 0.2),
    ('block2_conv2', 0.2),
#     ('block3_conv1', 0.2),
#     ('block3_conv2', 0.2),
#     ('block3_conv3', 0.2),
    ('block3_conv4', 0.2),
#     ('block4_conv1', 0.2),
#     ('block4_conv2', 0.2),
#     ('block4_conv3', 0.2),
    ('block4_conv4', 0.2),
#     ('block5_conv1', 0.2),
#     ('block5_conv2', 0.2),
#     ('block5_conv3', 0.2),
    ('block5_conv4', 0.2)]

In [74]:
class CONFIG:
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 0.025
    STYLE_WEIGHT = 5.0
    TOTAL_VARIATION_WEIGHT = 1.0
    NOISE = 0.6
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [75]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [76]:
def compute_content_loss(content, generated):
    m, n_H, n_W, n_C = generated.get_shape().as_list()

    content = tf.reshape(content, [-1])
    generated = tf.reshape(generated, [-1])

    return tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(generated, content)))) /\
                        (4*n_H*n_W*n_C)

In [77]:
def gram_matrix(x):
    gram = tf.matmul(x, x, transpose_b=True)
    return gram

def style_layer_loss(style, generated):
    m, n_H, n_W, n_C = generated.get_shape().as_list()
    
    style = tf.reshape(tf.transpose(style), (n_C, -1))
    generated = tf.reshape(tf.transpose(generated), (n_C, -1))
    
    S = gram_matrix(style)
    C = gram_matrix(generated)
    channels = n_C
    size = n_H*n_W

    return tf.reduce_sum(tf.square(tf.subtract(S, C))) \
            / (4. * (channels ** 2) * (size ** 2))

In [78]:
# Get content_image from file
content_image_path = "./images/louvre_small.jpg"
content_image = reshape_and_normalize_image(content_image_path)

image_to_show = plt.imread(content_image_path)
# plt.imshow(image_to_show)

In [79]:
style_image_path = "./images/monet.jpg"
style_image = reshape_and_normalize_image(style_image_path)

image_to_show = plt.imread(content_image_path)
# plt.imshow(image_to_show)

In [80]:
# Generate a random noise_image
generated_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT,
                                              CONFIG.IMAGE_WIDTH,
                                              CONFIG.COLOR_CHANNELS)).astype('float32')

# Set the input_image to be a weighted average of the content_image and a noise_image
generated_image = generated_image * CONFIG.NOISE + content_image * (1 - CONFIG.NOISE)

In [81]:
# Combined input for model
# input_tensor = K.concatenate([content_image,
#                           style_image,
#                           generated_image], axis=0)

input_tensor = tf.Variable(content_image, expected_shape=(1,300,400,3))
model = vgg19.VGG19(weights=None,
                    input_tensor=input_tensor,
                    include_top=False)
# model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

# model.summary()

In [82]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_list = [(layer_name, model.get_layer(layer_name).output) \
                     for layer_name, _ in STYLE_LAYERS]

layer_outputs = [output_tensors for _, output_tensors in outputs_list]
# print(layer_outputs)

In [83]:
model_input = model.get_layer("input_1").output

In [84]:
content_model = Model(inputs=model.input, outputs=layer_outputs)
content_model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [85]:
print(content_image.shape)

(1, 256, 256, 3)


In [86]:
# layer_outputs_content = model.predict(content_image)
# layer_outputs_style = model.predict(style_image)
# layer_outputs_generated = model.predict(generated_image)

In [88]:
# loss for content image
sess.run(model_input.assign(conten   t_image))

content_loss = 0.
content_features = sess.run(outputs_list[-2][1])
generated_features = outputs_list[-2][1]
content_loss = compute_content_loss(content_features, generated_features)

print(content_features)
print(type(content_features), content_features.shape)
print(generated_features)

[[[[   0.          499.9258728     0.         ...,    0.            0.
       0.        ]
   [   0.           63.74557114    0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.            0.
       0.        ]
   ..., 
   [   0.           63.13259125    0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.            0.
       0.        ]]

  [[   0.            0.            0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.            0.
       0.        ]
   ..., 
   [ 257.60629272  199.37548828    0.         ...,    0.            0.
       0.        ]
   [ 604.40631104  137.88998413    0.         ...,    0.            0.
       0.

In [19]:
# loss for style image
def compute_style_loss():
    # get the count of style layers
    count = len(STYLE_LAYERS)
    
    loss = 0.
    for layer_name, coeff in STYLE_LAYERS:
        # Select the output tensor of the currently selected layer
        out = outputs_list[-count][1]

        style_features = sess.run(out)
        generated_features = out
        sl = style_layer_loss(style_features, generated_features)
        loss += coeff * sl
    return loss

In [20]:
sess.run(model_input.assign(style_image))
style_loss = compute_style_loss()

# print(style_loss.eval())

In [21]:
def total_loss(content_loss, style_loss, alpha=10, beta=40):
    return (content_loss*alpha) + (style_loss*beta)

In [22]:
J = total_loss(content_loss, style_loss)
print(J.eval())

103546.0


In [23]:
# define optimizer
optimizer = tf.train.AdamOptimizer(2.0)

# define train_step
train_step = optimizer.minimize(J)

In [24]:
def save_image(path, image):
    
    # Un-normalize the image so that it looks good
    image = image + CONFIG.MEANS
    
    # Clip and Save the image
    image = np.clip(image[0], 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [25]:
def model_nn(sess, input_image, num_iterations = 3):
    
    # Initialize global variables (you need to run the session on the initializer)
    sess.run(tf.global_variables_initializer())
    
    sess.run(model_input.assign(input_image))
    
    for i in range(num_iterations):
        
        # Run the session on the train_step to minimize the total cost
        sess.run([train_step])
        
        # Compute the generated image by running the session on the current model['input']
        generated_image = sess.run(model_input)

        print(generated_image.shape)

        # Print every 20 iteration.
        if i%1 == 0:
            Jt, Jc, Js = sess.run([J, content_loss, style_loss])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
            # save current generated image in the "/output" directory
            save_image("output/" + str(i) + ".png", generated_image)
    
    # save last generated image
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image

In [26]:
model_nn(sess, generated_image)

(1, 256, 256, 3)
Iteration 0 :
total cost = inf
content cost = inf
style cost = 3.06153e+10


/Users/harshbhardwaj/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


(1, 256, 256, 3)
Iteration 1 :
total cost = nan
content cost = nan
style cost = nan
(1, 256, 256, 3)
Iteration 2 :
total cost = nan
content cost = nan
style cost = nan


array([[[[ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        ..., 
        [[ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         ..., 
         [ nan,  nan,  nan],
         [ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
  